# DoD Contradictions

This notebook demonstrates the same pipeline as from pipeline.py applied to the full DoD Issuances dataset. It was ran from an Azure Databricks environment.

In [0]:
# A little hack to get tqdm progress bars working.
#
# In the Databricks notebook, tqdm auto-detects a notebook but actually the std
# progress bar is desired! So, we just replace the tqdm.auto module with the
# tqdm.std module.
import sys
sys.modules['tqdm.auto'] = __import__('tqdm.std')  
sys.modules['tqdm.autonotebook'] = __import__('tqdm.std')

## Imports and Configuration

In [0]:
from src import loading, processing, custom_preprocessors, scoring
from functools import partial
from typing import Dict, List, Tuple

from haystack.schema import Document
import numpy as np
from pandas import DataFrame

In [0]:
# Which dataset to load.
DATASET_TABLE = "team_hdsiprodigies.contradictions_datasets_dod_issuances"
# How many documents to include in the pipeline run.
SUBSET_SIZE = None
# Parameters for cleaning sentences in initial chunking
CHUNK_CLEANING_TOC_PERIOD_THRESHOLD = 5
CHUNK_CLEANING_LENGTH_MINIMUM = 15
CHUNK_CLEANING_LENGTH_MAXIMUM = 1000
# Parameters for creating chunks
CHUNK_LENGTH = 8
CHUNK_OVERLAP = 2
# Parameters for pre-selecting similar chunks
EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'
MAX_CHUNK_SIMILARITY_THRESHOLD = 0.87
CHUNK_SIMILARITY_TOP_N = 500
# Parameters for selecting sentences as contradiction candidates
SENTENCE_CLEANING_LENGTH_MINIMUM = 40
CANDIDATE_SELECTION_TOP_K = 1000
SAVED_CHUNKS_FILEPATH = '/dbfs/FileStore/desired-chunks.pkl'
SAVED_CONTRADICTION_SCORES_FILEPATH = '/dbfs/FileStore/contradiction-scores.csv'
SAVED_CANDIDATES_FILEPATH = '/dbfs/FileStore/candidates.csv'


In [0]:
# -------------------------------- LOADING ------------------------------- #
df = loading.load_dataset_from_pyspark(DATASET_TABLE)


In [0]:
# ------------------------------ PREPROCESSING --------------------------- #
df['fulltext'] = df.text_by_page.apply(processing.clean_and_combine_pages)

if SUBSET_SIZE:
    df = df.iloc[:SUBSET_SIZE]
docs = processing.convert_frame_to_haystack(df)

# A chunk is constructed using a sliding window. It will be N sentences long
# if there are that many sentences remaining in the document. The next chunk
# will include the last K sentences of the previous chunk if a previous
# chunk exists. Chunks will not span documents.
chunk_sentence_cleaning_func = partial(processing.clean_sentence_splits,
    toc_period_threshold = CHUNK_CLEANING_TOC_PERIOD_THRESHOLD,
    length_minimum = CHUNK_CLEANING_LENGTH_MINIMUM,
    length_maximum = CHUNK_CLEANING_LENGTH_MAXIMUM,
)
chunker = custom_preprocessors.SplitCleanerPreProcessor(
    language='en',
    split_by='sentence',
    split_cleaner=chunk_sentence_cleaning_func,
    split_length=CHUNK_LENGTH,
    split_overlap=CHUNK_OVERLAP,
    split_respect_sentence_boundary=False, # incompatible with 'passage' or 'sentence'
)
doc_chunks = chunker.process(docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Unzipping tokenizers/punkt.zip.
Preprocessing: 0%| | 0/1124 [00:00<?, ?docs/s]Preprocessing: 100%|██████████| 1124/1124 [00:17<00:00, 65.79docs/s]

In [0]:
len(doc_chunks)

Out[7]: 52705

In [0]:
doc_chunks = processing.remove_identical_chunks(doc_chunks)

In [0]:
len(doc_chunks)

Out[9]: 52682

In [0]:
# --------------------------- CHUNK SIMILARITY --------------------------- #
# Available models: https://www.sbert.net/docs/pretrained_models.html
embeddings = processing.compute_chunk_embeddings(
    chunks=doc_chunks,
    model_name=EMBEDDING_MODEL_NAME,
    show_progress_bar=True,
)
# Enrich our chunks with embeddings
for chunk, embedding in zip(doc_chunks, embeddings):
    chunk.embedding = embedding

Downloading (…)e9125/.gitattributes: 0%| | 0.00/1.18k [00:00<?, ?B/s]Downloading (…)e9125/.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 143kB/s]
Downloading (…)_Pooling/config.json: 0%| | 0.00/190 [00:00<?, ?B/s]Downloading (…)_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 33.2kB/s]
Downloading (…)7e55de9125/README.md: 0%| | 0.00/10.6k [00:00<?, ?B/s]Downloading (…)7e55de9125/README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 3.24MB/s]
Downloading (…)55de9125/config.json: 0%| | 0.00/612 [00:00<?, ?B/s]Downloading (…)55de9125/config.json: 100%|██████████| 612/612 [00:00<00:00, 214kB/s]
Downloading (…)ce_transformers.json: 0%| | 0.00/116 [00:00<?, ?B/s]Downloading (…)ce_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 42.7kB/s]
Downloading (…)125/data_config.json: 0%| | 0.00/39.3k [00:00<?, ?B/s]Downloading (…)125/data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 7.20MB/s]
Downloading pytorch_model.bin: 0%| | 0.00/90.9M [00:00<?, ?B/s]Downloading pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:00<00:00, 199MB/s]
Downloading (…)nce_bert_config.json: 0%| | 0.00/53.0 [00:00<?, ?B/s]Downloading (…)nce_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 7.38kB/s]
Downloading (…)cial_tokens_map.json: 0%| | 0.00/112 [00:00<?, ?B/s]Downloading (…)cial_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 33.1kB/s]
Downloading (…)e9125/tokenizer.json: 0%| | 0.00/466k [00:00<?, ?B/s]Downloading (…)e9125/tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 20.2MB/s]
Downloading (…)okenizer_config.json: 0%| | 0.00/350 [00:00<?, ?B/s]Downloading (…)okenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 135kB/s]
Downloading (…)9125/train_script.py: 0%| | 0.00/13.2k [00:00<?, ?B/s]Downloading (…)9125/train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 4.83MB/s]
Downloading (…)7e55de9125/vocab.txt: 0%| | 0.00/232k [00:00<?, ?B/s]Downloading (…)7e55de9125/vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 10.8MB/s]
Downloading (…)5de9125/modules.json: 0%| | 0.00/349 [00:00<?, ?B/s]Downloading (…)5de9125/modules.json: 100%|██████████| 349/349 [00:00<00:00, 121kB/s]
/databricks/python/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly. To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
 return self.fget.__get__(instance, owner)()
Batches: 0%| | 0/1647 [00:00<?, ?it/s]Batches: 100%|██████████| 1647/1647 [12:24<00:00, 2.21it/s]

In [0]:
similarity_matrix = processing.compute_chunk_similarity(doc_chunks)

In [0]:
# Get rid of chunks that have similarity scores that are too high. This
# value was fine-tuned by trial and error to remove similar chunks that were
# just common header/disclaimer text.
top_n_pair_indices = processing.get_top_n_similar_chunk_pair_indices(
    scores=similarity_matrix,
    n=CHUNK_SIMILARITY_TOP_N,
    max_similarity_threshold=MAX_CHUNK_SIMILARITY_THRESHOLD,
)

In [0]:
similar_chunk_id_pairs = [
    (doc_chunks[c1].id, doc_chunks[c2].id) for c1, c2 in top_n_pair_indices
]
desired_indices = np.unique(np.array(top_n_pair_indices).flat)
# NOTE: Storing our desired chunks to a new variable should allow us to
# dispose of the full set of doc_chunks from memory, which we may need for
# the contradiction scoring model! If we need this, additional code will be
# required here.
desired_chunks = {
    doc_chunks[i].id: doc_chunks[i]
    for i in desired_indices
}

sentence_cleaning_func = partial(processing.clean_sentence_splits,
    length_minimum = SENTENCE_CLEANING_LENGTH_MINIMUM,
)
desired_chunk_sentences = processing.split_chunks_to_sentences(
    chunks=desired_chunks.values(),
    split_cleaner=sentence_cleaning_func,
)
# Enrich our desired chunks with sentences
for chunk, sentences in zip(desired_chunks.values(), desired_chunk_sentences):
    chunk.sentences = sentences

In [0]:
len(desired_chunks)

Out[14]: 918

In [0]:
# ------------------------- CONTRADICTION SCORING ------------------------ #
tokenizer, contradiction_model = scoring.load_contradiction_model()

Downloading (…)okenizer_config.json: 0%| | 0.00/49.0 [00:00<?, ?B/s]Downloading (…)okenizer_config.json: 100%|██████████| 49.0/49.0 [00:00<00:00, 12.2kB/s]
Downloading (…)lve/main/config.json: 0%| | 0.00/703 [00:00<?, ?B/s]Downloading (…)lve/main/config.json: 100%|██████████| 703/703 [00:00<00:00, 258kB/s]
Downloading (…)olve/main/vocab.json: 0%| | 0.00/899k [00:00<?, ?B/s]Downloading (…)olve/main/vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 27.2MB/s]
Downloading (…)olve/main/merges.txt: 0%| | 0.00/456k [00:00<?, ?B/s]Downloading (…)olve/main/merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 17.3MB/s]
Downloading (…)cial_tokens_map.json: 0%| | 0.00/772 [00:00<?, ?B/s]Downloading (…)cial_tokens_map.json: 100%|██████████| 772/772 [00:00<00:00, 266kB/s]
Downloading pytorch_model.bin: 0%| | 0.00/1.43G [00:00<?, ?B/s]Downloading pytorch_model.bin: 100%|██████████| 1.43G/1.43G [00:37<00:00, 38.2MB/s]
/databricks/python/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly. To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
 return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

In [0]:
contradiction_scores = scoring.compute_sentence_contradiction_scores(
    chunks=desired_chunks,
    chunk_id_pairs=similar_chunk_id_pairs,
    tokenizer=tokenizer,
    model=contradiction_model,
)

0%| | 0/500 [00:00<?, ?it/s] 50%|████▉ | 248/500 [18:42<24:34, 5.85s/it]100%|██████████| 500/500 [43:49<00:00, 5.26s/it]

In [0]:
# Save chunks and contradiction scores so that we can look at more than just the
# top k candidates in the future if we desire.
loading.save_candidates_csv(contradiction_scores, SAVED_CONTRADICTION_SCORES_FILEPATH)
loading.save_chunks_pickle(desired_chunks, SAVED_CHUNKS_FILEPATH)

# To download locally, modify the URL of this page so that it looks like
# https://adb-...azuredatabricks.net/files/FILENAME_UNDER_FileStore_DIRECTORY/?o=...
# keeping the id the same in the address and o= parameter. It will prompt you to
# save the file!

In [0]:
candidates = scoring.get_top_k_contradictive_candidates(
    contradiction_scores=contradiction_scores,
    k=CANDIDATE_SELECTION_TOP_K,
)
candidate_info = scoring.retrieve_candidate_info(
    candidates=candidates,
    chunks=desired_chunks
)

In [0]:
loading.save_candidates_csv(candidate_info, SAVED_CANDIDATES_FILEPATH)

In [0]:
candidate_info.head()

Out[19]:

,chunk_A.id,chunk_A.content,chunk_A.meta.corpus,chunk_A.meta.title,chunk_A.meta.file_name,chunk_A.meta.url,chunk_A.meta.id,chunk_A.meta._split_id,sentence_A,chunk_B.id,chunk_B.content,chunk_B.meta.corpus,chunk_B.meta.title,chunk_B.meta.file_name,chunk_B.meta.url,chunk_B.meta.id,chunk_B.meta._split_id,sentence_B,entailment,neutral,contradiction
0,a49093d345ebb00f1112e4082e976508,This data element is also known as Pay Entry Base Date. Format: YYYYMMDD. If...,dod_issuances,Automated Extract of Active Duty Military Personnel Records,DoDI 1336.05,https://www.esd.whs.mil/Portals/54/Documents/DD/issuances/dodi/133605p.pdf?v...,64966f2897774248e1a0baa7cbc55457f366d28f9cc02c8f537541706ad76214,32,22 292-294 Accession Program Source 292 a. Enlisted Accession Program Source...,e2a990ab455acef9aa6c415dab3c51f8,A Induction B Voluntary enlistment in a regular component C Voluntary enlist...,dod_issuances,Automated Extract of Active Duty Military Personnel Records,DoDI 1336.05,https://www.esd.whs.mil/Portals/54/Documents/DD/issuances/dodi/133605p.pdf?v...,64966f2897774248e1a0baa7cbc55457f366d28f9cc02c8f537541706ad76214,33,"Applicable only to commissioned officers, other than commissioned warrant of...",0.000045,0.000343,0.999613
1,c1feed26381cb1db40cf683aa8621e34,This instruction is available on the Directives Division Website at https://...,dod_issuances,"Military Officer Actions Requiring Presidential, Secretary of Defense, or Un...",DoDI 1320.04,https://www.esd.whs.mil/Portals/54/Documents/DD/issuances/dodi/132004p.pdf,3bc7f996335f339c23b84e1e6276aefc2f41cdf103bd541c9ac1043c0d631f09,3,"This instruction is effective January 3, 2014.",fb9ba053bced31cbee8703ba7edd427f,This Instruction is approved for public release and is available on the Dire...,dod_issuances,Voluntary Education Programs,DoDI 1322.25,https://www.esd.whs.mil/Portals/54/Documents/DD/issuances/dodi/132225p.pdf,0c1b0c20b979b9075e85264650aa4341a319a2c4c677f89045bd183bfc1d6f48,6,"This Instruction is effective March 15, 2011.",0.000046,0.000415,0.999538
2,31a0f2e22eee6ad67b2d84f89cd42e96,SUMMARY OF CHANGE 2. This administrative change updates: a. The title of the...,dod_issuances,Intelligence Support to the Defense Critical Infrastructure Program (DCIP),DoDI 3020.51,https://www.esd.whs.mil/Portals/54/Documents/DD/issuances/dodi/302051p.pdf?v...,70a017956d0d0962694db3ae7b44100fc64072f69ed56f6a3fa2c2b08dc35490,3,This administrative change updates: a. The title of the Under Secretary of D...,106946c53f00fd46e35bca5f47b5b5fa,This instruction is available on the Directives Division Website at https://...,dod_issuances,DoD Cryptologic Training,DoDI 3305.09,https://www.esd.whs.mil/Portals/54/Documents/DD/issuances/dodi/330509p.pdf?v...,15858382aaf9d25d3ed4dab9bba37c1aca2fbed3468410eb799b83875ad00eaf,3,This administrative change updates the title of the Under Secretary of Defen...,0.000126,0.000366,0.999508
3,a1bdd1d1e8e667ce28733f212889c882,Requires a 4-year service agreement: 2 years on active duty plus 2 years in ...,dod_issuances,Defense Manpower Data Center Domain Values for Military Personnel Data Extracts,DoDM 1336.05,https://www.esd.whs.mil/Portals/54/Documents/DD/issuances/dodm/133605m.PDF?v...,fb16fb843de46b5fbc89e4857e25ee5649efdc877e397d788fe609562b36241f,80,Requires a 3-year active duty service agreement.,9395790f7231ae64937e2b505771dada,Requires a 4-year active duty service agreement. D5 $150 Effective 1 August ...,dod_issuances,Defense Manpower Data Center Domain Values for Military Personnel Data Extracts,DoDM 1336.05,https://www.esd.whs.mil/Portals/54/Documents/DD/issuances/dodm/133605m.PDF?v...,fb16fb843de46b5fbc89e4857e25ee5649efdc877e397d788fe609562b36241f,85,Requires a 6-year active duty service agreement.,0.000058,0.000440,0.999502
4,ead7ad04914e16e33dd8baa9b5f63cfd,Requires a 4-year service agreement: 2 years on active duty plus 2 years in ...,dod_issuances,Defense Manpower Data Center Domain Values for Military Personnel Data Extracts,DoDM 1336.05,https://www.esd.wh

In [0]:
# Example of the first 100 results
for idx, candidate in candidate_info[:100].iterrows():
    print(f"({idx})")
    scoring.pretty_print_candidate(candidate)
    print('\n\n')

(0)
Title:	Automated Extract of Active Duty Military Personnel Records
-----
22 292-294 Accession Program Source 292 a. Enlisted Accession Program Source Code The code that represents the accession program by which a member first obtained enlisted status (also known as Means of Initial Entry into Military Service, Enlisted.)Applicable only to enlisted members.

Title:	Automated Extract of Active Duty Military Personnel Records
-----
Applicable only to commissioned officers, other than commissioned warrant officers.



(1)
Title:	Military Officer Actions Requiring Presidential, Secretary of Defense, or Under Secretary of Defense for Personnel and Readiness Approval or Senate Confirmation
-----
This instruction is effective January 3, 2014.

Title:	Voluntary Education Programs
-----
This Instruction is effective March 15, 2011.



(2)
Title:	Intelligence Support to the Defense Critical Infrastructure Program (DCIP)
-----
This administrative change updates: a. The title of the Under Secretary of Defense for Intelligence to the Under Secretary of Defense for Intelligence and Security (USD(I&S)) in accordance with Public Law 116-92 (Reference (l)), also known as the “National Defense Authorization Act for Fiscal Year 2020.”

Title:	DoD Cryptologic Training
-----
This administrative change updates the title of the Under Secretary of Defense for Intelligence to the Under Secretary of Defense for Intelligence and Security in accordance with Public Law 116-92 (Reference (d)).



(3)
Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 3-year active duty service agreement.

Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 6-year active duty service agreement.



(4)
Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 3-year active duty service agreement.

Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 6-year active duty service agreement.



(5)
Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 2-year active duty service agreement.

Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 6-year active duty service agreement.



(6)
Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 2-year active duty service agreement.

Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 6-year active duty service agreement.



(7)
Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 4-year active duty service agreement.

Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 6-year active duty service agreement.



(8)
Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 4-year active duty service agreement.

Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 6-year active duty service agreement.



(9)
Title:	Secretary of Defense Maintenance Awards
-----
DoDI 1348.30, April 8, 2019 SECTION 2: RESPONSIBILITIES 6 b. Forward, to the ASD(S), a maximum of two nomination packages per competition category (small, medium, and large) annually, during May, for deserving field-level units, for evaluation by the Secretary of Defense Field-Level Maintenance Awards Selection Board.

Title:	Secretary of Defense Maintenance Awards
-----
Requests each Combatant Command submit nominations annually, during January, for the Sustainment Train, Advise, and Assist Award within their area of responsibility.



(10)
Title:	Defense Manpower Data Center Domain Values for Military Personnel Data Extracts
-----
Requires a 3-year active duty service agreement.

Title